In [28]:
import torch_geometric
from torch_geometric.data import Data
import torch

import networkx as nx

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
anime = pd.read_csv("../data/anime.csv")
rating = pd.read_csv("../data/rating.csv")

anime['genre'] = anime['genre'].str.split(', ')
#anime = anime.dropna(axis=1)
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic, Mil...",TV,64,9.26,793665
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samurai, ...",TV,51,9.25,114262
3,9253,Steins;Gate,"[Sci-Fi, Thriller]",TV,24,9.17,673572
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samurai, ...",TV,51,9.16,151266


In [3]:
anime = anime.dropna()
anime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12017 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12017 non-null  int64  
 1   name      12017 non-null  object 
 2   genre     12017 non-null  object 
 3   type      12017 non-null  object 
 4   episodes  12017 non-null  object 
 5   rating    12017 non-null  float64
 6   members   12017 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 751.1+ KB


In [4]:
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [22]:
types = anime["type"].unique()
genres = anime["genre"].explode().unique()

type2id = {t:i for i, t in enumerate(types)}
id2type = {i:t for i, t in enumerate(types)}

genre2id = {g:i for i, g in enumerate(genres)}
id2genre = {i:g for i, g in enumerate(genres)}

unique_values = {
    "anime_id" : anime["anime_id"].unique(),
    "types" : [type2id[t] for t in anime["type"].unique()],
    "genre" : [genre2id[g] for g in anime["genre"].explode().unique()],
    "user_id": rating["user_id"].unique(),
}

In [23]:
G = nx.Graph()

G.add_nodes_from(unique_values["anime_id"])
G.add_nodes_from(unique_values["types"])
G.add_nodes_from(unique_values["genre"])
G.add_nodes_from(unique_values["user_id"])


for anime_id in unique_values["anime_id"]:
    G.nodes[anime_id]["rating"] = anime[anime["anime_id"] == anime_id]["rating"].values

for _, row in anime.iterrows():
    anime_id = row["anime_id"]
    
    anime_type = type2id[row["type"]]
    genres = [genre2id[g] for g in row["genre"]]

    G.add_edge(anime_id, anime_type, relation="type")
    
    for genre in genres:
        G.add_edge(anime_id, genre, relation="genre")

for _, row in rating.iterrows():
    user_id = row["user_id"]
    anime_id = row["anime_id"]
    rating_value = row["rating"]

    G.add_edge(user_id, anime_id, weight=rating_value, relation="rating")

In [25]:
edge_index = []
edge_type = []
for u, v, data in G.edges(data=True):
    edge_index.append([u, v])
    if data["relation"] == "rating":
        edge_type.append(0)
    elif data["relation"] == "type":
        edge_type.append(1)
    elif data["relation"] == "genre":
        edge_type.append(2)

edge_index = torch.tensor(edge_index, dtype=torch.long).T
edge_type = torch.tensor(edge_type, dtype=torch.long) 

In [ ]:
node_features = []
for node in G:
    if node in unique_values["anime_id"]:
        node_features.append(G.nodes[node]["rating"])
    elif node in unique_values["genre"]:
        node_features.append([0])
    elif node in unique_values["type"]:
        node_features.append([1])
    elif node in unique_values["user_id"]:
        node_features.append([2])

x = torch.tensor(node_features, dtype=torch.float).view(-1, 1)

/var/folders/v8/y55_th9545761gnl8qj270040000gn/T/ipykernel_84789/4195323992.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  x = torch.tensor(node_features, dtype=torch.float).view(-1, 1)


In [29]:
data = Data(x=x, edge_index=edge_index, edge_type=edge_type)
data

Data(x=[73516, 1], edge_index=[2, 7851481], edge_type=[7851481])